In [1]:
from urslib2 import RSS, SplitmmCIF, DSSR
import urslib2.SS.Relation as Relation
import os, glob

In [2]:
PDBPATH   = "./PDB/"
MOTIFPATH = "./refmotifs/"
HITFILE   = "unique_hits.tsv"
RESFILE   = "annotated_hits.tsv"

In [2]:
files  = sorted(glob.glob(os.path.join(PDBPATH,"*.cif")))
motifs = sorted(glob.glob(os.path.join(MOTIFPATH,"*")))
len(files), len(motifs)

(7256, 2)

In [3]:
scheme = "-2b -2n -1b -1n L1 L1p 1n 1b 2b 2n 3b 3n 4b 4n".split()

cores = {"1ffk_0_kt7.cif": { "L1":"0.G.94.", "L1p":None,
                            "-1b":"0.C.93.", "-1n":"0.G.81.", 
                            "-2b":"0.G.92.", "-2n":"0.C.82.", 
                             "1n":"0.A.80.",  "1b":"0.G.97.",
                             "2b":"0.A.98.",  "2n":"0.G.79.",
                             "3b":"0.A.99.",  "3n":"0.G.78.",
                             "4b":"0.C.100.", "4n":"0.G.77."},
         
         "3d2g_A_kj.cif":  { "L1":"A.C.38.", "L1p":"A.G.8.",
                            "-1b":"A.C.37.", "-1n":"A.G.9.", 
                            "-2b":"A.C.36.", "-2n":"A.G.10.",
                             "1n":"A.A.72.",  "1b":"A.G.42.",
                             "2b":"A.A.44.",  "2n":"A.G.71.",
                             "3b":"A.C.45.",  "3n":"A.G.70.",
                             "4b":"A.C.46.",  "4n":"A.G.69."},
        }

In [4]:

hits = {}

with open(HITFILE) as inp:
        lines = inp.readlines()
        for line in lines[1:]:
            if line.strip():
                linesplit = line.strip().split()
                pdbfile = linesplit[-2]
                if pdbfile not in hits:
                    hits[pdbfile] = []
                hits[pdbfile].append(line)
len(hits)

2110

In [ ]:
def AtomDist(a1,a2):
    return (a1['X']-a2['X'])**2 + (a1['Y']-a2['Y'])**2 + (a1['Z']-a2['Z'])**2

cnt = 0
outp = open(RESFILE,'w')

title = ['PDB', "MOL", 'REFERENCE','SIZE','RMSD'] +\
        scheme + [x+'BASE' for x in scheme] + [x+'SS' for x in scheme] +\
        ["XwayJunction","RANGE","KINK","TYPE"]

outp.write("\t".join(title)+'\n')

for file in files:

    if file not in hits:
        continue

    pdbmodel  = file.replace(".cif",".cif1")
    dssrout   = pdbmodel.replace(".cif",".out")
    
    if not os.path.exists(pdbmodel):
        SplitmmCIF.Into_models(file, PDBPATH)
    if not os.path.exists(dssrout):
        DSSR.run(pdbmodel, PDBPATH)
    
    model   = RSS.SecStruct(pdbmodel, dssrout)
    cnt += 1
    pdb = os.path.basename(pdbmodel)
    print(pdb, cnt)
    juncd = {}
    for loop in model.loops['JUNCTION']:
        xway = loop['THREADSNUM']
        if not loop['PTYPE'] == 'C':
            continue
        for t in loop['TLOOP']:
            thread = model.threads[t['THREAD']-1]
            if thread['LEN']:
                for nucl in model.chains[thread['CHAIN']][thread['START'][1]]\
                                        [thread["START"][2]:thread["END"][2]+1]:
                    juncd[nucl['DSSR']] = xway
    
    for line in hits[file]:

        linesplit = line.strip().split()
        size, rmsd = linesplit[1], linesplit[2]
        ref = os.path.basename(linesplit[-1])
        match = (pair.split('=') for pair in linesplit[-3].split(','))
        match = {y.split('.',1)[1]:x.split('.',1)[1] for x,y in match}
        core = {token:match[cores[ref][token]] 
                if cores[ref][token] and cores[ref][token] in match
                else ''
                for token in scheme}
        coress = {token:model.NuclSS(core[token]) if core[token] else '' for token in core}
                    
        xway = set()
        for dssr in core.values():
            if dssr and dssr in juncd:
                xway.add(juncd[dssr])
        xway = ','.join([str(x) for x in sorted(xway)])
                    
        lclr = "NA"
        if core['L1'] and core['1n'] and core['2b']:
            if model.NuclRelation(core['L1'],core['1n']) == 'LR' and\
                model.NuclRelation(core['L1'],core['2b']) == 'LR':
                lclr = 'LR'
            else:
                lclr = 'LC'
                    
        kink = 'NA'
        if core['L1'] and core['2b']:
            if 0 <= model.SeqDist(core['L1'],core['2b']) < 10:
                kink = 'YES'
            else:
                kink = 'NO'
                    
        Type = 'NA'
        if core['L1'] and core['1n']:
            n1 = Relation.GetNuclByDSSR(model,core['L1'])
            n2 = Relation.GetNuclByDSSR(model,core['1n'])
            if n2['NAME'] in {'A','G'}:       
                try:
                    atom1 = [atom for atom in n1['ATOMS'] if atom['NAME']=="O2'"][0]
                    atom2 = [atom for atom in n2['ATOMS'] if atom['NAME']=="N1"][0]
                    atom3 = [atom for atom in n2['ATOMS'] if atom['NAME']=="N3"][0]
                    if AtomDist(atom1,atom2) <= AtomDist(atom1,atom3):
                        Type = "N1"
                    else:
                        Type = "N3"
                except:
                    pass
                            
        mols = {model.molecules[model.chains[dssr.split('.')[0]]['MOL_ID']]['MOLECULE'] 
                for dssr in core.values() if dssr}
        mols = ','.join(sorted(mols))
                    
        res = [pdb,mols,ref,size,rmsd]
        res += [core[t] for t in scheme]
        res += [core[t].split('.')[1] if core[t] else '' for t in scheme]
        res += [coress[t] for t in scheme]
        res += [xway, lclr, kink, Type]
        outp.write("\t".join([str(x) for x in res])+'\n')
                            
outp.close()              

1FFK.cif1 1
1FJG.cif1 2
1GRZ.cif1 3
1HNW.cif1 4
1HNX.cif1 5
1HNZ.cif1 6
1HR0.cif1 7
1I94.cif1 8
1I95.cif1 9
1I96.cif1 10
1I97.cif1 11
1IBK.cif1 12
1IBL.cif1 13
1IBM.cif1 14
1J5A.cif1 15
1J5E.cif1 16
1JJ2.cif1 17
1JZX.cif1 18
1JZY.cif1 19
1JZZ.cif1 20
1K01.cif1 21
1K73.cif1 22
1K8A.cif1 23
1K9M.cif1 24
1KC8.cif1 25
1KD1.cif1 26
1KQS.cif1 27
1M1K.cif1 28
1M90.cif1 29
1N32.cif1 30
1N33.cif1 31
1N34.cif1 32
1N36.cif1 33
1N8R.cif1 34
1NJI.cif1 35
1NJM.cif1 36
1NJN.cif1 37
1NJO.cif1 38
1NJP.cif1 39
1NKW.cif1 40
1NWX.cif1 41
1NWY.cif1 42
1OND.cif1 43
1P9X.cif1 44
1Q7Y.cif1 45
1Q81.cif1 46
1Q82.cif1 47
1Q86.cif1 48
1QVF.cif1 49
1QVG.cif1 50
1S72.cif1 51
1SM1.cif1 52
1T0K.cif1 53
1VQ4.cif1 54
"DNA OH 5 prime terminus" is inappropriate type of residue
1VQ5.cif1 55
1VQ6.cif1 56
1VQ7.cif1 57
1VQ8.cif1 58
1VQ9.cif1 59
1VQK.cif1 60
1VQL.cif1 61
1VQM.cif1 62
1VQN.cif1 63
1VQO.cif1 64
"DNA OH 5 prime terminus" is inappropriate type of residue
1VQP.cif1 65
1VVJ.cif1 66
1VY4.cif1 67
1VY5.cif1 68
1VY6.ci